In [1]:
%cd ..

a:\Phong\Crawl_buffett


In [7]:
# import shutil
# import pandas as pd

# df = pd.read_csv('docs\miss_irbank.csv')
# for symbol in df['0']:
#     shutil.rmtree(f'Data/{symbol}', ignore_errors=True)


In [3]:
import pandas as pd
import os
import numpy as np
df_1 = pd.read_csv('docs/May_anhNA.csv')
df_2 = pd.read_csv('docs/may_cty.csv')

In [4]:
df = pd.read_csv('docs/List_company_23052023 - Listing.csv')
# df['check'] = np.nan
# df['volume'] = np.nan
# df['dividend'] = np.nan
# df['table'] = np.nan
for id in df.index:
    symbol = df['Symbol'][id]
    if os.path.exists(fr'Data\{symbol}\docs\link.csv'):
        df['check'][id] = 'Done'
        if os.path.exists(fr'Data\{symbol}\docs\volume.csv'):
            df['volume'][id] = 'Done'
        if os.path.exists(fr'Data\{symbol}\docs\dividend.csv'):
            df['dividend'][id] = 'Done'
        if os.path.exists(fr'Data\{symbol}\docs\checklist_get_table.csv'):
            df['table'][id] = 'Done'
df.to_csv('docs/List_company_23052023 - Listing.csv', index = False)

In [ ]:
df = pd.read_csv('docs\may_cty.csv')
# df = pd.read_csv('docs\May_anhNA.csv')

df['check'] = np.nan
df['volume'] = np.nan
df['dividend'] = np.nan
df['table'] = np.nan
for id in df.index:
    symbol = df['Symbol'][id]
    if os.path.exists(fr'Data\{symbol}\docs\link.csv'):
        df['check'][id] = 'Done'
        if os.path.exists(fr'Data\{symbol}\docs\volume.csv'):
            df['volume'][id] = 'Done'
        if os.path.exists(fr'Data\{symbol}\docs\dividend.csv'):
            df['dividend'][id] = 'Done'
        if os.path.exists(fr'Data\{symbol}\docs\checklist_get_table.csv'):
            df['table'][id] = 'Done'
    else:
        df['check'][id] = np.nan
# df.to_csv('docs\May_anhNA.csv', index = False)
df.to_csv('docs\may_cty.csv', index = False)


In [6]:
lst_miss = []
df = pd.read_csv('docs\may_cty.csv')
for id in df.index:
    symbol = df['Symbol'][id]
    if os.path.exists(fr'Data\{symbol}\docs\link.csv') and os.path.exists(fr'Data\{symbol}\docs\check.csv') == False:
        lst_miss.append(symbol)
# df.to_csv('docs\miss_irbank.csv', index = False)

pd.DataFrame(lst_miss).to_csv('docs\miss_irbank.csv', index = False)

# move file

In [3]:
%cd ..
import pandas as pd
import numpy as np
import os

a:\Phong\Crawl_buffett


In [ ]:
import shutil
PATH_DRIVER = "G:/My Drive/6_2023/Financial_1/"
PATH_IS = PATH_DRIVER + 'IncomeStatement/'
PATH_BS = PATH_DRIVER + 'BalanceSheet/'
os.makedirs(PATH_IS, exist_ok=True)
os.makedirs(PATH_BS, exist_ok=True)

symbol = 1301
def moveFinan(symbol):
    path_bs = rf'Data\{symbol}\table_bs'
    path_pl = rf'Data\{symbol}\table_pl'
    for file in os.listdir(path_bs):
        if file.endswith('Q4.csv'):
            os.makedirs(PATH_BS + '/' + str(symbol), exist_ok=True)
            shutil.copy(path_bs + '/' + file, PATH_BS + '/' + str(symbol) + '/' + file)
    for file in os.listdir(path_pl):
        if file.endswith('Q4.csv'):
            os.makedirs(PATH_IS + str(symbol), exist_ok=True)
            shutil.copy(path_pl + '/' + file, PATH_IS + '/' + str(symbol) + '/' + file)

df = pd.read_csv('docs\List_company_23052023 - Listing.csv')
for id in df.index:
    symbol = df['Symbol'][id]
    if df['check'][id] == 'Done':
        try:
            moveFinan(symbol)
        except Exception as e:
            print(symbol, e)

In [ ]:
# PATH_DRIVER = "H:/My Drive/6_2023/Done_Japan1"
from datetime import timedelta


PATH_DRIVER = "I:/My Drive/6_2023/DoneJapan"
os.makedirs(rf'{PATH_DRIVER}/dividend', exist_ok=True)

def sortDate(df, type_ = 'volume'):
    if type_ == 'volume':
        df['Time'] = pd.to_datetime(df['Time'], format='%Y_%m_%d', errors='coerce')
    else:
        df['Time'] = pd.to_datetime(df['Time'], format='%d_%m_%Y', errors='coerce')
    df.sort_values(by=['Time'], inplace=True)
    df['Time'] = df['Time'].dt.strftime('%d/%m/%Y')
    return df

def delDate(time, date):
    if pd.isna(date):
        return date
    if 'Q1' in time or 'Q2' in time:
        date = pd.to_datetime(date, format='%Y_%m_%d', errors='coerce')
        date = date - timedelta(days=365)
        date = date.strftime('%Y_%m_%d')
    return date

def getVolume(symbol, PATH = rf'{PATH_DRIVER}/volume/'):
    df = pd.read_csv(fr'Data\{symbol}\docs\volume.csv')
    if not os.path.exists(fr'Data\{symbol}\docs\check.csv'):
        df['date'] = df[['time', 'date']].apply(lambda x: delDate(x['time'], x['date']), axis=1)
    df['volume'] = df['vol1'] - df['vol2']
    df = df[['date', 'volume']]
    df.dropna(how = 'all', inplace=True)
    df.rename(columns={'date': 'Time', 'volume': 'Volume'}, inplace=True)
    df = sortDate(df, type_ = 'volume')
    df.to_csv(fr'{PATH}{symbol}.csv', index = False)
    return df

def getDividend(symbol, PATH = rf'{PATH_DRIVER}/dividend/'):
    df = pd.read_csv(fr'Data\{symbol}\docs\dividend.csv')
    df_done = pd.DataFrame(columns = ['Time', 'Stock', 'Money', 'Time2'])
    df_done['Time'] = df[['time_split_Q1', 'time_split_Q2', 'time_split_Q3', 'time_split_Q4']].stack(dropna=False).reset_index(drop=True)
    df_done['Time'] = df_done['Time'].str.replace("['", '').str.replace("']", '')
    df_done['Money'] = df[['Q1', 'Q2', 'Q3', 'Q4']].stack(dropna=False).reset_index(drop=True)
    for q in  ['Q1', 'Q2', 'Q3', 'Q4']:
        df[q] = df['Year'].astype(str) + '_' + q
    df_done['Time2'] = df[['Q1', 'Q2', 'Q3', 'Q4']].stack(dropna=False).reset_index(drop=True)
    df_done.replace('－', np.nan, inplace=True)
    df_done.replace('―', np.nan, inplace=True)
    df_done.replace('-', np.nan, inplace=True)
    df_done.replace('—', np.nan, inplace=True)
    df_done.replace('一', np.nan, inplace=True)
    df_done.replace('=', np.nan, inplace=True)
    df_done.dropna(inplace=True, thresh=2)
    df_done = sortDate(df_done, type_ = 'dividend')
    df_done.to_csv(fr'{PATH}{symbol}.csv', index = False)
    return df_done

symbol = 2190
getVolume(symbol)
getDividend(symbol)

In [3]:
df = pd.read_csv('docs\List_company_23052023 - Listing.csv')
for id in df.index:
    symbol = df['Symbol'][id]
    if df['check'][id] == 'Done':
        if os.path.exists(fr'Data\{symbol}\docs\volume.csv'):
            try:
                getVolume(symbol)
            except Exception as e:
                print(symbol, e)
                break

        if os.path.exists(fr'Data\{symbol}\docs\dividend.csv'):
            try:
                getDividend(symbol)
            except Exception as e:
                print(symbol, e)
                break


# volume

In [1]:
%cd ..
import pandas as pd
import numpy as np
import os
from src.volume import GetVolume

a:\Phong\Crawl_buffett


In [ ]:
vol = GetVolume(path_save='')
# vol.getDataFromPdf(id_company=1333, year = 2015, quy = 'Q1')
vol.getVolume(8333)

In [34]:
symbol = 2190
path_volume = fr'Data\{symbol}\docs\volume.csv'
path_link = fr'Data\{symbol}\docs\link.csv'
path1 = fr'Data\{symbol}\PDF'
df = pd.read_csv(path_volume)
df

,time,date,vol1,vol2
0,2015_Q1,2015_08_05,NaN,NaN
1,2014_Q1,NaN,NaN,NaN
2,2015_Q2,2015_11_06,NaN,NaN
3,2014_Q2,2014_11_08,NaN,NaN
4,2015_Q3,2015_02_05,4171300.0,385.0
5,2014_Q3,2014_02_07,NaN,NaN
6,2015_Q4,NaN,NaN,NaN
7,2014_Q4,2014_05_08,NaN,NaN


In [35]:
# df.apply(lambda x: delDate(x['time'], x['date']), axis=1)
df[['date']] = df.apply(lambda x: delDate(x['time'], x['date']), axis=1)
df

ValueError: Columns must be same length as key

In [3]:
from src.ocrPdf import ocrPDF

ocrPDF(fr"A:\Phong\Crawl_buffett\Data\5289\PDF\2017_Q1_(2017_08_10).pdf", 'hi.pdf')

EncryptedPdfError: Input PDF is encrypted. The encryption must be removed to
perform OCR.

For information about this PDF's security use
    qpdf --show-encryption infilename

You can remove the encryption using
    qpdf --decrypt [--password=[password]] infilename
